In [1]:
# add source to apt
# sudo apt update
# sudo apt install ni-daqmx
import time
import datetime
import pprint
import nidaqmx
import threading
import numpy as np
import nidaqmx.stream_writers

In [2]:
# 1 kHz, in chunks of 100
rate = 1000
samples_per_read = 100
# 10 Hz
#rate = 10
#samples_per_read = 10
samples = 0
def aquire_as_thread():
    with nidaqmx.Task() as task:
        task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
        task.ai_channels.add_ai_voltage_chan("Dev1/ai1")
        task.ai_channels.add_ai_voltage_chan("Dev1/ai2")

        task.timing.cfg_samp_clk_timing(rate, sample_mode=nidaqmx.constants.AcquisitionType.CONTINUOUS)

        def callback(task_handle, every_n_samples_event_type,
                     number_of_samples, callback_data):
            #print('Every N Samples callback invoked.')
            data = task.read(number_of_samples_per_channel=samples_per_read)
            if len(data)<10: # if more than one channel
                with open('nilog.txt','a') as fi:
                    logstr = str(datetime.datetime.now())+'\n'
                    for d in data:
                        logstr += ''.join([f'{s:.5f} ' for s in d]) +'\n'
                    fi.write(logstr)
            else: # if one channel
                with open('nilog.txt','a') as fi:
                    fi.write(str(datetime.datetime.now())+'\n'+ ''.join([f'{s:.5f} ' for s in data]) +'\n')
            global samples
            samples +=1
            return 0

        task.register_every_n_samples_acquired_into_buffer_event(
            samples_per_read, callback)

        task.start()            
        while keep_alive:
            print(f' {samples}x{samples_per_read} datapoints', end = '\r')
            time.sleep(1)
            
keep_alive = True 
threading.Thread(target=aquire_as_thread, args=()).start()


In [50]:
# to stop reading thread:
if 0:
    keep_alive = False
    print(f' {samples}x{samples_per_read} datapoints')

 2701x100 datapoints


sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x1fdf5d69e40>


In [3]:
import os
def get_last_lines(file, n, sep = ' / '):
    with open(file, 'rb') as f:
        if os.path.getsize(file) > 20000:
            f.seek(-20000, 2)
        data = f.readlines()
        lines = []
        for i in range(n,0,-1):
            lines.append(data[-i].decode("utf-8"))
        lines = sep.join(lines).replace('\r\n', '').strip()
    return lines

In [4]:
lines = get_last_lines('nilog.txt', 4, '\n')
print(lines)

2022-09-15 13:16:46.235479
-0.01842 -0.01874 -0.01842 -0.01858 -0.01842 -0.01874 -0.01842 -0.01826 -0.01826 -0.01826 -0.01826 -0.01793 -0.01809 -0.01793 -0.01809 -0.01793 -0.01826 -0.01809 -0.01826 -0.01826 -0.01858 -0.01826 -0.01842 -0.01858 -0.01809 -0.01842 -0.01842 -0.01842 -0.01842 -0.01826 -0.01809 -0.01826 -0.01826 -0.01793 -0.01809 -0.01793 -0.01809 -0.01842 -0.01842 -0.01826 -0.01842 -0.01826 -0.01826 -0.01842 -0.01842 -0.01826 -0.01842 -0.01826 -0.01842 -0.01809 -0.01793 -0.01793 -0.01793 -0.01809 -0.01809 -0.01793 -0.01826 -0.01842 -0.01842 -0.01826 -0.01826 -0.01858 -0.01890 -0.01874 -0.01858 -0.01842 -0.01842 -0.01826 -0.01826 -0.01842 -0.01809 -0.01793 -0.01793 -0.01809 -0.01793 -0.01793 -0.01793 -0.01809 -0.01826 -0.01809 -0.01826 -0.01826 -0.01842 -0.01858 -0.01842 -0.01842 -0.01858 -0.01826 -0.01858 -0.01826 -0.01842 -0.01777 -0.01793 -0.01793 -0.01809 -0.01793 -0.01809 -0.01826 -0.01842 -0.01826 
0.00025 0.00009 0.00009 0.00041 0.00025 -0.00007 0.00009 0.00009 -0.0000

In [11]:
lines = get_last_lines('nilog.txt', 16, '\n')
for l in lines.split('\n')[0::4]:
    print(l)

2022-09-15 13:17:30.339241
2022-09-15 13:17:30.435354
2022-09-15 13:17:30.536078
2022-09-15 13:17:30.636207
